# 06 Calculate Distance
- input: `data/05_augment_thai.csv`

In [1]:
import os
import pickle

import pandas as pd
import numpy as np

from utils import clean_text
from tqdm.notebook import tqdm
from scipy.spatial.distance import cosine

tqdm.pandas()

In [2]:
data = pd.read_parquet('data/05_augment_thai.parquet')
data

,question,answers,source,id,en_aug,th_aug,context,en_llm_gec_aug,en_llm_paraphrase_aug,en_qcpg_0.2_aug,...,th_qcpg_0.5_aug,th_qcpg_0.8_aug,th_qcpg_0.2_llm_gec_aug,th_qcpg_0.5_llm_gec_aug,th_qcpg_0.8_llm_gec_aug,th_wordnet_aug,th_thai2fit_aug,th_ltw2v_aug,th_thai2trans_aug,th_fasttext_aug
0,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,Phatthira Sarutpong Phokin What is the date of...,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the date of birth of Phatthira Sarutpo...,"Phatthira Sarutpong Phokin, could you please p...",What is the date of birth of phuket khan jr?,...,วันเกิดของภูเก็ตข่านคืออะไร?,วันเดือนปีเกิดของ Phattin Sarutin คืออะไร?,วันเกิดของภูเก็ตข่านคืออะไร?,phatthira sarutpah ng คืออะไร,วันเดือนปีเกิดของ Phatthira Sarutin คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันเท่าไร ?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ เกิดวันที่เท่าไ...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่่ ?
1,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,Phatthira Sarutpong Phokin What are you playing?,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"What are you playing, Phatthira Sarutpong Phokin?","Phatthira Sarutpong Phokin, what game are you ...",What are you playing when your sarutina is pla...,...,คุณกำลังเล่นอะไรเมื่อคุณเป็น Sarutina?,คุณเล่นอะไรกับ Phatthira Sarutong?,คุณกำลังเล่นอะไรเมื่อคุณเป็น Sarutin?,คุณกำลังเล่นอะไรเมื่อคุณเป็น phatthira sarut,คุณกำลังเล่นอะไรใน Phatthira Sarutong pH,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร ?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร??,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...
2,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,What is the career of Phatthira Teerathiyapong...,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the career of Phatthira Teerathiyapong...,What is Phatthira Teerathiyapong Phokin's prof...,What is the career of a physicist?,...,Phatthira Teeratha Phikin เส้นทางอาชีพอะไร,อาชีพของ phatthira teeratha phi คืออะไร,อาชีพนักฟิสิกส์คืออะไร?,Phatthira Teeratha Phikin เส้นทางอาชีพอะไร,อาชีพของ phatthira teeratha phi คืออะไร,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร ?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร??,พัทธ์ธีรา ศรุติพงศ์โภคิน ทําอาชีพอะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4 ?
3,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,Phatthira Sarutpong Phokin Graduated from what...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,From which country did Phatthira Sarutpong Pho...,From which country did Phatthira Sarutpong Pho...,Phatthira sarutpah phoenix graduated,...,phatthira sarutpah pH ประเทศอะไร,Phatthira Sarutpong Phokin จบการศึกษาจาก?,phatthira sarutpah pH ปีอะไร,phatthira sarutpah pH ปีอะไร,Phatthira Sarutpong Phokin ประเทศอะไร,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการอบรมจากประเทศอะไร ?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ จบการศึกษาจากปร...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...
4,บิดาของคลีโอพัตราเป็นใคร?,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,Who is Cleopatra's father?,พ่อของคลีโอพัตราคือใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,Who is the father of Cleopa

In [3]:
# Clean all question columns
cols = [col for col in data.columns if col.startswith('th_')]
for col in tqdm(cols):
    data[col] = data[col].progress_apply(lambda x: clean_text(x, is_question=True))

data

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

/home/parin/Projects/cross-lingual-data-augmentation-for-qa-in-low-resource-languages/questions/utils.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')


  0%|          | 0/16980 [00:00<?, ?it/s]

/home/parin/Projects/cross-lingual-data-augmentation-for-qa-in-low-resource-languages/questions/utils.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')


...

  0%|          | 0/16980 [00:00<?, ?it/s]

/home/parin/Projects/cross-lingual-data-augmentation-for-qa-in-low-resource-languages/questions/utils.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')


  0%|          | 0/16980 [00:00<?, ?it/s]

/home/parin/Projects/cross-lingual-data-augmentation-for-qa-in-low-resource-languages/questions/utils.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'lxml')


,question,answers,source,id,en_aug,th_aug,context,en_llm_gec_aug,en_llm_paraphrase_aug,en_qcpg_0.2_aug,...,th_qcpg_0.5_aug,th_qcpg_0.8_aug,th_qcpg_0.2_llm_gec_aug,th_qcpg_0.5_llm_gec_aug,th_qcpg_0.8_llm_gec_aug,th_wordnet_aug,th_thai2fit_aug,th_ltw2v_aug,th_thai2trans_aug,th_fasttext_aug
0,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,Phatthira Sarutpong Phokin What is the date of...,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the date of birth of Phatthira Sarutpo...,"Phatthira Sarutpong Phokin, could you please p...",What is the date of birth of phuket khan jr?,...,วันเกิดของภูเก็ตข่านคืออะไร?,วันเดือนปีเกิดของ Phattin Sarutin คืออะไร?,วันเกิดของภูเก็ตข่านคืออะไร?,phatthira sarutpah ng คืออะไร?,วันเดือนปีเกิดของ Phatthira Sarutin คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันเท่าไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ เกิดวันที่เท่าไ...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?
1,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,Phatthira Sarutpong Phokin What are you playing?,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"What are you playing, Phatthira Sarutpong Phokin?","Phatthira Sarutpong Phokin, what game are you ...",What are you playing when your sarutina is pla...,...,คุณกำลังเล่นอะไรเมื่อคุณเป็น Sarutina?,คุณเล่นอะไรกับ Phatthira Sarutong?,คุณกำลังเล่นอะไรเมื่อคุณเป็น Sarutin?,คุณกำลังเล่นอะไรเมื่อคุณเป็น phatthira sarut?,คุณกำลังเล่นอะไรใน Phatthira Sarutong pH?,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...
2,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,What is the career of Phatthira Teerathiyapong...,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the career of Phatthira Teerathiyapong...,What is Phatthira Teerathiyapong Phokin's prof...,What is the career of a physicist?,...,Phatthira Teeratha Phikin เส้นทางอาชีพอะไร?,อาชีพของ phatthira teeratha phi คืออะไร?,อาชีพนักฟิสิกส์คืออะไร?,Phatthira Teeratha Phikin เส้นทางอาชีพอะไร?,อาชีพของ phatthira teeratha phi คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?
3,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,Phatthira Sarutpong Phokin Graduated from what...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,From which country did Phatthira Sarutpong Pho...,From which country did Phatthira Sarutpong Pho...,Phatthira sarutpah phoenix graduated,...,phatthira sarutpah pH ประเทศอะไร?,Phatthira Sarutpong Phokin จบการศึกษาจาก?,phatthira sarutpah pH ปีอะไร?,phatthira sarutpah pH ปีอะไร?,Phatthira Sarutpong Phokin ประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการอบรมจากประเทศอะไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ จบการศึกษาจากปร...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...
4,บิดาของคลีโอพัตราเป็นใคร?,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,Who is Cleopatra's father?,พ่อของคลีโอพัตราคือใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,Who is the father of Cleop

In [4]:
# Sanity check for nan
data.isna().sum()

question                    0
answers                     0
source                      0
id                          0
en_aug                      0
th_aug                      0
context                    34
en_llm_gec_aug              0
en_llm_paraphrase_aug       0
en_qcpg_0.2_aug             1
en_qcpg_0.5_aug             1
en_qcpg_0.8_aug             3
en_qcpg_0.2_llm_gec_aug     0
en_qcpg_0.5_llm_gec_aug     0
en_qcpg_0.8_llm_gec_aug     0
th_llm_gec_aug              0
th_llm_paraphrase_aug       0
th_qcpg_0.2_aug             0
th_qcpg_0.5_aug             0
th_qcpg_0.8_aug             0
th_qcpg_0.2_llm_gec_aug     0
th_qcpg_0.5_llm_gec_aug     0
th_qcpg_0.8_llm_gec_aug     0
th_wordnet_aug              0
th_thai2fit_aug             0
th_ltw2v_aug                0
th_thai2trans_aug           0
th_fasttext_aug             0
dtype: int64

In [4]:
# The empty context is a result of google sheets quirks, so we will replace it with the one from step 01
# We can see that there are 34 missing values in the `context` column. We can fill it up with the original context from the first file by removing and merging.
original = pd.read_csv('data/01_prepare_dataset.csv')
data = data.drop(columns=["context"])
data = data.merge(original[["id", "context"]], on="id")
data

,question,answers,source,id,en_aug,th_aug,en_llm_gec_aug,en_llm_paraphrase_aug,en_qcpg_0.2_aug,en_qcpg_0.5_aug,...,th_qcpg_0.8_aug,th_qcpg_0.2_llm_gec_aug,th_qcpg_0.5_llm_gec_aug,th_qcpg_0.8_llm_gec_aug,th_wordnet_aug,th_thai2fit_aug,th_ltw2v_aug,th_thai2trans_aug,th_fasttext_aug,context
0,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,Phatthira Sarutpong Phokin What is the date of...,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,What is the date of birth of Phatthira Sarutpo...,"Phatthira Sarutpong Phokin, could you please p...",What is the date of birth of phuket khan jr?,What is the birth date of phuket khan?,...,วันเดือนปีเกิดของ Phattin Sarutin คืออะไร?,วันเกิดของภูเก็ตข่านคืออะไร?,phatthira sarutpah ng คืออะไร?,วันเดือนปีเกิดของ Phatthira Sarutin คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันเท่าไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ เกิดวันที่เท่าไ...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...
1,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,Phatthira Sarutpong Phokin What are you playing?,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,"What are you playing, Phatthira Sarutpong Phokin?","Phatthira Sarutpong Phokin, what game are you ...",What are you playing when your sarutina is pla...,What are you playing when you are sarutina?,...,คุณเล่นอะไรกับ Phatthira Sarutong?,คุณกำลังเล่นอะไรเมื่อคุณเป็น Sarutin?,คุณกำลังเล่นอะไรเมื่อคุณเป็น phatthira sarut?,คุณกำลังเล่นอะไรใน Phatthira Sarutong pH?,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...
2,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,What is the career of Phatthira Teerathiyapong...,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,What is the career of Phatthira Teerathiyapong...,What is Phatthira Teerathiyapong Phokin's prof...,What is the career of a physicist?,What career path did phatthira teeratha phikin,...,อาชีพของ phatthira teeratha phi คืออะไร?,อาชีพนักฟิสิกส์คืออะไร?,Phatthira Teeratha Phikin เส้นทางอาชีพอะไร?,อาชีพของ phatthira teeratha phi คืออะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...
3,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,Phatthira Sarutpong Phokin Graduated from what...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,From which country did Phatthira Sarutpong Pho...,From which country did Phatthira Sarutpong Pho...,Phatthira sarutpah phoenix graduated,What country did phatthira sarutpah ph,...,Phatthira Sarutpong Phokin จบการศึกษาจาก?,phatthira sarutpah pH ปีอะไร?,phatthira sarutpah pH ปีอะไร?,Phatthira Sarutpong Phokin ประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการอบรมจากประเทศอะไร?,พิชญ์ธีรา วิ่งผลัดพงศ์เบญจวรรณ จบการศึกษาจากปร...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...
4,บิดาของคลีโอพัตราเป็นใคร?,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,Who is Cleopatra's father?,พ่อของคลีโอพัตราคือใคร?,Who is the father of Cleopatra?,Who is Cleopatra's bi

In [5]:
qcpg = pd.read_csv('data/04_paraphrase_qcpg.csv')
data = data.drop(columns=["en_qcpg_0.2_aug", "en_qcpg_0.5_aug", "en_qcpg_0.8_aug", "en_qcpg_0.2_llm_gec_aug","en_qcpg_0.5_llm_gec_aug","en_qcpg_0.8_llm_gec_aug"])
data = data.merge(qcpg[["id", "en_qcpg_0.2_aug", "en_qcpg_0.5_aug", "en_qcpg_0.8_aug", "en_qcpg_0.2_llm_gec_aug","en_qcpg_0.5_llm_gec_aug","en_qcpg_0.8_llm_gec_aug"]], on="id")
data

,question,answers,source,id,en_aug,th_aug,en_llm_gec_aug,en_llm_paraphrase_aug,th_llm_gec_aug,th_llm_paraphrase_aug,...,th_ltw2v_aug,th_thai2trans_aug,th_fasttext_aug,context,en_qcpg_0.2_aug,en_qcpg_0.5_aug,en_qcpg_0.8_aug,en_qcpg_0.2_llm_gec_aug,en_qcpg_0.5_llm_gec_aug,en_qcpg_0.8_llm_gec_aug
0,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,Phatthira Sarutpong Phokin What is the date of...,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,What is the date of birth of Phatthira Sarutpo...,"Phatthira Sarutpong Phokin, could you please p...",วันเดือนปีเกิดของ Phatthira Sarutpong Phokin ค...,Phatthira Sarutpong Phokin คุณช่วยจัดเตรียมวัน...,...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the date of birth of phuket khan jr?,What is the birth date of phuket khan?,What is the date of birth of phattin sarutin?,What is the birth date of phuket khan?,What is phatthira sarutpah ng date of,What is the date of birth of phatthira sarutin?
1,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,Phatthira Sarutpong Phokin What are you playing?,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,"What are you playing, Phatthira Sarutpong Phokin?","Phatthira Sarutpong Phokin, what game are you ...",คุณกำลังเล่นอะไร Phatthira Sarutpong Phokin?,Phatthira Sarutpong Phokin คุณกำลังเล่นเกมอะไร...,...,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What are you playing when your sarutina is pla...,What are you playing when you are sarutina?,What are you playing with phatthira sarutong?,What are you playing when you are a sarutin?,What are you playing when you are a phatthira ...,What are you playing in phatthira sarutong ph
2,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,What is the career of Phatthira Teerathiyapong...,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,What is the career of Phatthira Teerathiyapong...,What is Phatthira Teerathiyapong Phokin's prof...,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,What is the career of a physicist?,What career path did phatthira teeratha phikin,What is the career of phatthira teeratha phi,What is the career of a physicist?,What career path did phatthira teeratha phikin,What is the career of phatthira teeratha phi
3,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,Phatthira Sarutpong Phokin Graduated from what...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,From which country did Phatthira Sarutpong Pho...,From which country did Phatthira Sarutpong Pho...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศใด?,Phatthira Sarutpong Phokin จากประเทศใดจากการศึ...,...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,Phatthira sarutpah phoenix graduated,What country did phatthira sarutpah ph,Phatthira sarutpong phokin graduated from?,What year did phatthira sarutpah ph,What year did phatthira sarutpah ph,What country did phatthira sarutpong phokin
4,บิดาของคลีโอพัตราเป็นใคร?,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,50c66c986cdf19b60f36da1b89137

In [6]:
assert data.isna().sum().sum() == 0

In [7]:
data.columns

Index(['question', 'answers', 'source', 'id', 'en_aug', 'th_aug',
       'en_llm_gec_aug', 'en_llm_paraphrase_aug', 'th_llm_gec_aug',
       'th_llm_paraphrase_aug', 'th_qcpg_0.2_aug', 'th_qcpg_0.5_aug',
       'th_qcpg_0.8_aug', 'th_qcpg_0.2_llm_gec_aug', 'th_qcpg_0.5_llm_gec_aug',
       'th_qcpg_0.8_llm_gec_aug', 'th_wordnet_aug', 'th_thai2fit_aug',
       'th_ltw2v_aug', 'th_thai2trans_aug', 'th_fasttext_aug', 'context',
       'en_qcpg_0.2_aug', 'en_qcpg_0.5_aug', 'en_qcpg_0.8_aug',
       'en_qcpg_0.2_llm_gec_aug', 'en_qcpg_0.5_llm_gec_aug',
       'en_qcpg_0.8_llm_gec_aug'],
      dtype='object')

In [8]:
# Sort all the columns properly
all_cols = data.columns

sorted_cols = ['id', 'question', 'context', 'answers', 'source'] + sorted([col for col in all_cols if col.startswith('th_')]) + sorted([col for col in all_cols if col.startswith('en_')])
assert len(all_cols) == len(sorted_cols)

sorted_cols

['id',
 'question',
 'context',
 'answers',
 'source',
 'th_aug',
 'th_fasttext_aug',
 'th_llm_gec_aug',
 'th_llm_paraphrase_aug',
 'th_ltw2v_aug',
 'th_qcpg_0.2_aug',
 'th_qcpg_0.2_llm_gec_aug',
 'th_qcpg_0.5_aug',
 'th_qcpg_0.5_llm_gec_aug',
 'th_qcpg_0.8_aug',
 'th_qcpg_0.8_llm_gec_aug',
 'th_thai2fit_aug',
 'th_thai2trans_aug',
 'th_wordnet_aug',
 'en_aug',
 'en_llm_gec_aug',
 'en_llm_paraphrase_aug',
 'en_qcpg_0.2_aug',
 'en_qcpg_0.2_llm_gec_aug',
 'en_qcpg_0.5_aug',
 'en_qcpg_0.5_llm_gec_aug',
 'en_qcpg_0.8_aug',
 'en_qcpg_0.8_llm_gec_aug']

In [9]:
data = data[sorted_cols].copy()
data

,id,question,context,answers,source,th_aug,th_fasttext_aug,th_llm_gec_aug,th_llm_paraphrase_aug,th_ltw2v_aug,...,th_wordnet_aug,en_aug,en_llm_gec_aug,en_llm_paraphrase_aug,en_qcpg_0.2_aug,en_qcpg_0.2_llm_gec_aug,en_qcpg_0.5_aug,en_qcpg_0.5_llm_gec_aug,en_qcpg_0.8_aug,en_qcpg_0.8_llm_gec_aug
0,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,วันเดือนปีเกิดของ Phatthira Sarutpong Phokin ค...,Phatthira Sarutpong Phokin คุณช่วยจัดเตรียมวัน...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันเท่าไร?,Phatthira Sarutpong Phokin What is the date of...,What is the date of birth of Phatthira Sarutpo...,"Phatthira Sarutpong Phokin, could you please p...",What is the date of birth of phuket khan jr?,What is the birth date of phuket khan?,What is the birth date of phuket khan?,What is phatthira sarutpah ng date of,What is the date of birth of phattin sarutin?,What is the date of birth of phatthira sarutin?
1,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,คุณกำลังเล่นอะไร Phatthira Sarutpong Phokin?,Phatthira Sarutpong Phokin คุณกำลังเล่นเกมอะไร...,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,Phatthira Sarutpong Phokin What are you playing?,"What are you playing, Phatthira Sarutpong Phokin?","Phatthira Sarutpong Phokin, what game are you ...",What are you playing when your sarutina is pla...,What are you playing when you are a sarutin?,What are you playing when you are sarutina?,What are you playing when you are a phatthira ...,What are you playing with phatthira sarutong?,What are you playing in phatthira sarutong ph
2,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,What is the career of Phatthira Teerathiyapong...,What is the career of Phatthira Teerathiyapong...,What is Phatthira Teerathiyapong Phokin's prof...,What is the career of a physicist?,What is the career of a physicist?,What career path did phatthira teeratha phikin,What career path did phatthira teeratha phikin,What is the career of phatthira teeratha phi,What is the career of phatthira teeratha phi
3,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศใด?,Phatthira Sarutpong Phokin จากประเทศใดจากการศึ...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการอบรมจากประเทศอะไร?,Phatthira Sarutpong Phokin Graduated from what...,From which country did Phatthira Sarutpong Pho...,From which country did Phatthira Sarutpong Pho...,Phatthira sarutpah phoenix graduated,What year did phatthira sarutpah ph,What country did phatthira sarutpah ph,What year did phatthira sarutpah ph,Phatthira sarutpong phokin graduated from?,What country did phatthira sarutpong phokin
4,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,บิดาของคลีโอพัตราเป็นใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρ

# Use mUSE to calculate the distances
Please download the model from here: https://github.com/mrpeerat/CL-ReLKT
- Go to `models`
- Select the `xquad` folder
- Download the `student_best_supported_languages.zip` folder as a zip file

In [10]:
# !unzip student_best_supported_languages.zip

In [11]:
import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
from utils import encode_in_batch

2023-08-29 17:04:23.594019: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-29 17:04:23.616769: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 17:04:24.322446: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
# Calculate Distance
def calculate_distance(model, df, col_name):
    all_cols = df.columns

    original_questions_embed = encode_in_batch(model, df['question'].tolist())

    # If the column has an english counterpart perform harmonic distance
    if col_name.replace('th_', 'en_') in all_cols:
        th_embed = encode_in_batch(model, df[col_name].tolist())
        en_embed = encode_in_batch(model, df[col_name.replace('th_', 'en_')].tolist())
    
        # Compute the cosine distance
        dis_to_en = []
        dis_to_th = []
        for a, b in zip(original_questions_embed, th_embed):
            dis_to_th.append(cosine(a, b))

        for a, b in zip(original_questions_embed, en_embed):
            dis_to_en.append(cosine(a, b))
        
        # Harmonic distance
        # https://en.wikipedia.org/wiki/Harmonic_mean
        dis_to_en = np.asarray(dis_to_en)
        dis_to_th = np.asarray(dis_to_th)
        return (2 * dis_to_en * dis_to_th) / (dis_to_en + dis_to_th) 
    else:
        # If the column has no english counterpart perform cosine distance
        embed = encode_in_batch(model, df[col_name].tolist())
        dis_to_th = []
        for a, b in zip(original_questions_embed, embed):
            dis_to_th.append(cosine(a, b))

        return dis_to_th

In [13]:
all_augment_cols = [col for col in data.columns if col.startswith('th_')]
all_augment_cols

['th_aug',
 'th_fasttext_aug',
 'th_llm_gec_aug',
 'th_llm_paraphrase_aug',
 'th_ltw2v_aug',
 'th_qcpg_0.2_aug',
 'th_qcpg_0.2_llm_gec_aug',
 'th_qcpg_0.5_aug',
 'th_qcpg_0.5_llm_gec_aug',
 'th_qcpg_0.8_aug',
 'th_qcpg_0.8_llm_gec_aug',
 'th_thai2fit_aug',
 'th_thai2trans_aug',
 'th_wordnet_aug']

In [14]:
model_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

2023-08-29 17:05:22.215499: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-29 17:05:22.250811: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [15]:
for col in tqdm(all_augment_cols):
    dis_col_name = col.replace('th_', 'dis_')
    data.loc[:,dis_col_name] = calculate_distance(model_embed, data, col)

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

...

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

  0%|          | 0/133 [00:00<?, ?it/s]

In [16]:
data

,id,question,context,answers,source,th_aug,th_fasttext_aug,th_llm_gec_aug,th_llm_paraphrase_aug,th_ltw2v_aug,...,dis_ltw2v_aug,dis_qcpg_0.2_aug,dis_qcpg_0.2_llm_gec_aug,dis_qcpg_0.5_aug,dis_qcpg_0.5_llm_gec_aug,dis_qcpg_0.8_aug,dis_qcpg_0.8_llm_gec_aug,dis_thai2fit_aug,dis_thai2trans_aug,dis_wordnet_aug
0,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,วันเดือนปีเกิดของ Phatthira Sarutpong Phokin ค...,Phatthira Sarutpong Phokin คุณช่วยจัดเตรียมวัน...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,...,0.379822,0.457964,0.442132,0.442132,0.664607,0.373971,0.391374,0.213284,0.000000,9.606189e-02
1,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,คุณกำลังเล่นอะไร Phatthira Sarutpong Phokin?,Phatthira Sarutpong Phokin คุณกำลังเล่นเกมอะไร...,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,...,0.028428,0.512295,0.604564,0.552465,0.590820,0.583206,0.645776,0.232190,0.000000,0.000000e+00
2,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,...,0.017806,0.408607,0.408607,0.346143,0.346143,0.328919,0.328919,0.150410,0.000000,0.000000e+00
3,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศใด?,Phatthira Sarutpong Phokin จากประเทศใดจากการศึ...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,...,0.044393,0.600420,0.717404,0.606477,0.717404,0.433692,0.523367,0.155829,0.009204,1.820072e-01
4,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,บิดาของคลีโอพัตราเป็นใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,พ่อของคลีโอพัตราคือใคร?,บิดาของมเนสโอพัตตกฺขหลายๆใคร?,พ่อของคลีโอพัตราคือใคร?,พ่อผู้ให้กำเนิดของคลีโอพัตราคือใคร?,มารดาของคลีโอพัตราเป็นใคร?,...,0.180941,0.238710,0.388979,0.151006,0.443043,0.097685,0.097685,0.150196,0.156331,8.621460e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16975,e780cc4812dd88f21ec7f0152905b15588d1a6faab66d2...,หนังสือการ์ตูนชานะ นักรบเนตรอัคคี มีกี่เล่ม?,นิยาย ชานะ นักรบเนตรอัคคี แต่งโดย ยาชิจิโร ทาค...,"{'text': ['22', '26'], 'answer_start': [267, 9...",tydiqa,หนังสือการ์ตูนนักรบดับเพลิงกี่คน?,หนังสือDIYการ์ตูนPokemonชานะ อุดมการณ์ถ.ทิพยเน...,นักรบดับเพลิงมีกี่คนในหนังสือการ์ตูน?,ในหนังสือการ์ตูนมีนักรบดับเพลิงกี่คน?,หนังสือการ์ตูนชาน่ะ นักรบเนตรอัคคี มีกี่เล่ม?,...,0.038205,0.459041,0.345795,0.330124,0.345795,0.363513,0.284700,0.063899,0.012518,1.700147e-01
16976,dd424979403f3909773a832c4d83818e564fe7bfeb4f38...,ไบโอช็อก อินฟินิต เปิดตัวครั้งแรกเมื่อไหร่?,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,"{'text': ['26 มีนาคม พ.ศ. 2556', '26 มีนาคม พ....",tydiqa,BioShock Infin เปิดตัวครั้งแรกเมื่อใด?,ไบโอมีช๊อค อร์เนชยมีฟิตี้07ต เปิดตัวครั้งแรกเม...,BioShock Infinite เปิดตัวครั้งแรกเมื่อใด?,การเปิดตัวครั้งแรกของ BioShock Infinite คือเมื...,ไบโอช็อค อินฟินิต เผยโฉมเป็นครั้งแรกเมื่อไร?,...,0.168088,0.609325,0.713147,0.475034,0.686884,0.482104,0.602809,0.097

In [20]:
data.to_parquet("data/06_calculate_distance.parquet", index=False)

# Calculate BLEU score

In [1]:
import pandas as pd
from pythainlp.tokenize import word_tokenize, sent_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_parquet("data/06_calculate_distance.parquet")
data

,id,question,context,answers,source,th_aug,th_fasttext_aug,th_llm_gec_aug,th_llm_paraphrase_aug,th_ltw2v_aug,...,dis_llm_paraphrase_aug,dis_ltw2v_aug,dis_qcpg_0.2_aug,dis_qcpg_0.2_llm_gec_aug,dis_qcpg_0.5_aug,dis_qcpg_0.5_llm_gec_aug,dis_qcpg_0.8_aug,dis_qcpg_0.8_llm_gec_aug,dis_thai2fit_aug,dis_wordnet_aug
0,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,วันเดือนปีเกิดของ Phatthira Sarutpong Phokin ค...,Phatthira Sarutpong Phokin คุณช่วยจัดเตรียมวัน...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,...,0.654346,0.333436,0.942305,0.901118,0.901118,0.783473,0.677738,0.642690,0.595411,0.224245
1,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,คุณกำลังเล่นอะไร Phatthira Sarutpong Phokin?,Phatthira Sarutpong Phokin คุณกำลังเล่นเกมอะไร...,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,...,0.600163,0.285477,0.712166,0.668647,0.792416,0.537381,0.611034,0.867701,0.387706,0.000000
2,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,...,0.690551,0.227241,0.896181,0.896181,0.617513,0.617513,0.750872,0.750872,0.392775,0.000000
3,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศใด?,Phatthira Sarutpong Phokin จากประเทศใดจากการศึ...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,...,0.659828,0.469348,0.723428,0.849254,0.870112,0.849254,0.528563,0.648129,0.376103,0.276969
4,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,บิดาของคลีโอพัตราเป็นใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,พ่อของคลีโอพัตราคือใคร?,บิดาของมเนสโอพัตตกฺขหลายๆใคร?,พ่อของคลีโอพัตราคือใคร?,พ่อผู้ให้กำเนิดของคลีโอพัตราคือใคร?,มารดาของคลีโอพัตราเป็นใคร?,...,0.630858,0.627320,0.716229,0.672312,0.727237,0.751295,0.794606,0.794606,0.574593,0.631798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16975,e780cc4812dd88f21ec7f0152905b15588d1a6faab66d2...,หนังสือการ์ตูนชานะ นักรบเนตรอัคคี มีกี่เล่ม?,นิยาย ชานะ นักรบเนตรอัคคี แต่งโดย ยาชิจิโร ทาค...,"{'text': ['22', '26'], 'answer_start': [267, 9...",tydiqa,หนังสือการ์ตูนนักรบดับเพลิงกี่คน?,หนังสือDIYการ์ตูนPokemonชานะ อุดมการณ์ถ.ทิพยเน...,นักรบดับเพลิงมีกี่คนในหนังสือการ์ตูน?,ในหนังสือการ์ตูนมีนักรบดับเพลิงกี่คน?,หนังสือการ์ตูนชาน่ะ นักรบเนตรอัคคี มีกี่เล่ม?,...,0.911251,0.195957,0.880506,0.930000,0.899868,0.930000,0.910610,0.905050,0.312825,0.522812
16976,dd424979403f3909773a832c4d83818e564fe7bfeb4f38...,ไบโอช็อก อินฟินิต เปิดตัวครั้งแรกเมื่อไหร่?,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,"{'text': ['26 มีนาคม พ.ศ. 2556', '26 มีนาคม พ....",tydiqa,BioShock Infin เปิดตัวครั้งแรกเมื่อใด?,ไบโอมีช๊อค อร์เนชยมีฟิตี้07ต เปิดตัวครั้งแรกเม...,BioShock Infinite เปิดตัวครั้งแรกเมื่อใด?,การเปิดตัวครั้งแรกของ BioShock Infinite คือเมื...,ไบโอช็อค อินฟินิต เผยโฉมเป็นครั้งแรกเมื่อไร?,...,0.880787,0.476029,0.900869,0.820951,0.922494,0.922002,0.901534,0.925271,0.458451,0.00000

In [3]:
all_augment_cols = [col for col in data.columns if col.startswith('th_')]
all_augment_cols

['th_aug',
 'th_fasttext_aug',
 'th_llm_gec_aug',
 'th_llm_paraphrase_aug',
 'th_ltw2v_aug',
 'th_qcpg_0.2_aug',
 'th_qcpg_0.2_llm_gec_aug',
 'th_qcpg_0.5_aug',
 'th_qcpg_0.5_llm_gec_aug',
 'th_qcpg_0.8_aug',
 'th_qcpg_0.8_llm_gec_aug',
 'th_thai2fit_aug',
 'th_wordnet_aug']

In [4]:
from joblib import Parallel, delayed

def calculate_single_bleu(original, generated):
    return round(
        sentence_bleu(
            [word_tokenize(original, keep_whitespace=False, engine="attacut")],
            word_tokenize(generated, keep_whitespace=False, engine="attacut"),
            smoothing_function=SmoothingFunction().method1,
        ),
        9
    )

def calculate_bleu(df, col_name, n_jobs=-1):
    original = df['question'].to_list()
    generated = df[col_name].to_list()

    bleu = Parallel(n_jobs=n_jobs)(
        delayed(calculate_single_bleu)(o, g) for o, g in tqdm(zip(original, generated), total=len(original))
    )

    return bleu

for col in tqdm(all_augment_cols):
    bleu_col_name = col.replace('th_', 'bleu_')
    data.loc[:, bleu_col_name] = calculate_bleu(data, col)


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

...

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

In [5]:
data

,id,question,context,answers,source,th_aug,th_fasttext_aug,th_llm_gec_aug,th_llm_paraphrase_aug,th_ltw2v_aug,...,bleu_llm_paraphrase_aug,bleu_ltw2v_aug,bleu_qcpg_0.2_aug,bleu_qcpg_0.2_llm_gec_aug,bleu_qcpg_0.5_aug,bleu_qcpg_0.5_llm_gec_aug,bleu_qcpg_0.8_aug,bleu_qcpg_0.8_llm_gec_aug,bleu_thai2fit_aug,bleu_wordnet_aug
0,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,วันเดือนปีเกิดของ Phatthira Sarutpong Phokin ค...,Phatthira Sarutpong Phokin คุณช่วยจัดเตรียมวัน...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,...,0.018798,0.056122,0.027776,0.036556,0.036556,0.029252,0.027776,0.027776,0.277762,0.515449
1,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,คุณกำลังเล่นอะไร Phatthira Sarutpong Phokin?,Phatthira Sarutpong Phokin คุณกำลังเล่นเกมอะไร...,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,...,0.027776,0.707107,0.027776,0.031560,0.031560,0.027776,0.038498,0.036556,0.516973,1.000000
2,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,...,0.065006,0.537285,0.102950,0.102950,0.202052,0.202052,0.065006,0.065006,0.345721,1.000000
3,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศใด?,Phatthira Sarutpong Phokin จากประเทศใดจากการศึ...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,...,0.049133,0.392815,0.111472,0.044325,0.103737,0.044325,0.284695,0.103737,0.570675,0.658037
4,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,บิดาของคลีโอพัตราเป็นใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,พ่อของคลีโอพัตราคือใคร?,บิดาของมเนสโอพัตตกฺขหลายๆใคร?,พ่อของคลีโอพัตราคือใคร?,พ่อผู้ให้กำเนิดของคลีโอพัตราคือใคร?,มารดาของคลีโอพัตราเป็นใคร?,...,0.049394,0.759836,0.043989,0.027776,0.048549,0.039281,0.103052,0.103052,0.205567,0.537285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16975,e780cc4812dd88f21ec7f0152905b15588d1a6faab66d2...,หนังสือการ์ตูนชานะ นักรบเนตรอัคคี มีกี่เล่ม?,นิยาย ชานะ นักรบเนตรอัคคี แต่งโดย ยาชิจิโร ทาค...,"{'text': ['22', '26'], 'answer_start': [267, 9...",tydiqa,หนังสือการ์ตูนนักรบดับเพลิงกี่คน?,หนังสือDIYการ์ตูนPokemonชานะ อุดมการณ์ถ.ทิพยเน...,นักรบดับเพลิงมีกี่คนในหนังสือการ์ตูน?,ในหนังสือการ์ตูนมีนักรบดับเพลิงกี่คน?,หนังสือการ์ตูนชาน่ะ นักรบเนตรอัคคี มีกี่เล่ม?,...,0.058739,0.631555,0.027585,0.057061,0.054115,0.057061,0.024089,0.120670,0.534174,0.137471
16976,dd424979403f3909773a832c4d83818e564fe7bfeb4f38...,ไบโอช็อก อินฟินิต เปิดตัวครั้งแรกเมื่อไหร่?,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,"{'text': ['26 มีนาคม พ.ศ. 2556', '26 มีนาคม พ....",tydiqa,BioShock Infin เปิดตัวครั้งแรกเมื่อใด?,ไบโอมีช๊อค อร์เนชยมีฟิตี้07ต เปิดตัวครั้งแรกเม...,BioShock Infinite เปิดตัวครั้งแรกเมื่อใด?,การเปิดตัวครั้งแรกของ BioShock Infinite คือเมื...,ไบโอช็อค อินฟินิต เผยโฉมเป็นครั้งแรกเมื่อไร?,...,0.042836,0.127534,0.163481,0.017033,0.060307,0.023980,0.434721,0.028518,0.4347

In [6]:
# Semantic Lexical Evaluation Metric
def calculate_single_slem(distance, bleu_score, weight_distance=1, weight_bleu=1, normalize=True):
    score = (weight_distance * distance) - (weight_bleu * bleu_score)

    if normalize:
        score = (score + 1) / 2
    
    return score

def calculate_slem(df, col_name, weight_distance=1, weight_bleu=1, normalize=True):
    dis_col_name = col_name.replace('th_', 'dis_')
    bleu_col_name = col_name.replace('th_', 'bleu_')

    distances = df[dis_col_name].to_list()
    bleu_scores = df[bleu_col_name].to_list()

    slem_scores = Parallel(n_jobs=-1)(delayed(calculate_single_slem)(distance, bleu_score, weight_distance, weight_bleu, normalize) for distance, bleu_score in tqdm(zip(distances, bleu_scores), total=len(distances)))

    return slem_scores

for col in tqdm(all_augment_cols):
    slem_col_name = col.replace('th_', 'slem_')
    data.loc[:, slem_col_name] = calculate_slem(data, col)


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

...

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

  0%|          | 0/16980 [00:00<?, ?it/s]

In [24]:
data[["question", "dis_llm_gec_aug", "slem_llm_gec_aug", "bleu_llm_gec_aug", "th_llm_gec_aug"]][data["bleu_llm_gec_aug"] != 1.0].sort_values(by="slem_llm_gec_aug", ascending=False).head(30)

,question,dis_llm_gec_aug,slem_llm_gec_aug,bleu_llm_gec_aug,th_llm_gec_aug
10676,ผู้ป่วยโรคนาร์ซิสซัสซินโดรมจะมีอาการใด?,1.153803,1.062178,0.029447,อาการของ Narcissus syndrome คืออะไร?
5313,อาณาจักรโคกูรยอโดนกองทัพใดตีแตก?,1.111208,1.047836,0.015537,อาณาจักรของ Kokuro ได้รับผลกระทบจากกองทัพหรือไม่?
11316,เพย์ตัน แมนนิง พากี่ทีมไปสู่ซูเปอร์โบว์ล?,1.108543,1.045033,0.018477,ทีมของ Peyton Manning ทำให้มันเป็น Super Bowl ...
6696,โรงเรียนบางระจันวิทยาก่อตั้งขึ้นเมื่อไร?,1.097335,1.038115,0.021105,วันที่จัดตั้งโรงเรียน Bang Rachan Witthaya คือ...
16091,ใครแสดงเป็นตัวละครที่ชื่อว่า เป็นต่อ ในซิตคอม ...,1.099843,1.037780,0.024283,ใครเล่นตัวละครในซิทคอม?
12505,จังหวัดปราจีนบุรีตั้งอยู่ภาคใดของไทย?,1.108866,1.029736,0.049394,จังหวัด Prachin Buri ตั้งอยู่ที่ไหนในประเทศไทย?
11737,สิ่งที่คอสตัลเบรอยด์ไม่มีเหมือนทีโนฟอราชนิดอื่...,1.079953,1.028606,0.022741,อะไรทำให้ Costalbaid แตกต่างจาก Tanovrands อื่...
12534,บีทีเอสเป็นวงที่ก่อตั้งขึ้นเมื่อวันที่เท่าไหร่?,1.098662,1.028147,0.042368,BTS BTS ก่อตั้งขึ้นในวันใด?
2604,ทะเลลีกูเรียนเป็นทะเลขนาดเล็กในทะเลใด?,1.073389,1.027150,0.019090,ทะเลเล็ก ๆ ใกล้ Lee Kurang ชื่ออะไร?
16806,วัดหนองป่าพงตั้งอยู่ที่จังหวัดอะไร?,1.113207,1.026450,0.060307,วัด Pa Pong ตั้งอยู่ในจังหวัดใด?


In [25]:
data.to_parquet("data/06_calculate_distance.parquet", index=False)